# Concours sur les prix de vente immobiliers

Pour implémenter la validation croisée, tester différents paramètres et trouver la meilleure valeur de paramètre, nous allons suivre plusieurs étapes. Voici comment procéder pour améliorer votre code en intégrant ces éléments :

1. **Création de la fonction utile** : Nous allons définir une fonction pour exécuter la validation croisée avec un modèle et des paramètres donnés.

2. **Tester différentes valeurs de paramètres** : Utiliser un ensemble de paramètres pour entraîner et évaluer les performances du modèle sur différentes configurations.

3. **Trouver la meilleure valeur de paramètre** : Sélectionner la valeur qui donne les meilleures performances, en utilisant la validation croisée pour éviter le surapprentissage.

Voici comment vous pouvez intégrer cela à votre code :


In [4]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Charger les données
train_path = 'train.csv'  # Remplacez par le chemin correct
X_full = pd.read_csv(train_path, index_col='Id')

test_path = 'test.csv'  # Remplacez par le chemin correct
X_test_full = pd.read_csv(test_path, index_col='Id')

# Prétraitement initial
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full['SalePrice']
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Séparer les données d'entraînement et de validation
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    X_full, y, train_size=0.8, test_size=0.2, random_state=0
)

# Identifier les colonnes numériques et catégoriques
categorical_cols = [
    cname for cname in X_train_full.columns
    if X_train_full[cname].dtype == "object" and X_train_full[cname].nunique() < 10
]
numerical_cols = [
    cname for cname in X_train_full.columns
    if X_train_full[cname].dtype in ['int64', 'float64']
]

# Garder les colonnes sélectionnées
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# Définir les transformations pour les colonnes numériques et catégoriques
numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combiner les transformations dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Définir le modèle
model = RandomForestRegressor(random_state=0)

# Créer le pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Fonction pour effectuer la validation croisée avec GridSearchCV
def cross_validate_model(pipeline, X_train, y_train, param_grid):
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_

# Définir la grille des hyperparamètres à tester
param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10]
}

# Appliquer la validation croisée pour trouver le meilleur modèle
best_model, best_params, best_score = cross_validate_model(my_pipeline, X_train, y_train, param_grid)

# Afficher les résultats
print("Meilleur modèle :", best_model)
print("Meilleurs paramètres :", best_params)
print("Meilleur score MAE :", -best_score)



ModuleNotFoundError: No module named 'sklearn'

In [6]:
# Entraîner le meilleur modèle sur l'ensemble d'entraînement complet
best_model.fit(X_train, y_train)

# Faire des prédictions sur les données de validation
preds_valid = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, preds_valid)
print(f'MAE sur les données de validation : {mae}')

# Faire des prédictions sur les données de test
preds_test = best_model.predict(X_test)

# Préparer le fichier de soumission
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('submission_best_model.csv', index=False)
print("Fichier de soumission créé : submission_best_model.csv")

MAE sur les données de validation : 17438.391436895043
Fichier de soumission créé : submission_best_model.csv


### Explication des modifications :

1. **Fonction `cross_validate_model`** : Cette fonction utilise `GridSearchCV` pour effectuer la recherche par grille sur les hyperparamètres spécifiés dans `param_grid`. Elle effectue une validation croisée avec 5 plis (`cv=5`) et renvoie le meilleur modèle, les meilleurs paramètres et le meilleur score.

2. **Recherche par grille** : La grille des hyperparamètres inclut différentes valeurs pour `n_estimators`, `max_depth`, et `min_samples_split` dans le modèle `RandomForestRegressor`. Ces paramètres sont ajustés pour trouver la combinaison qui minimise l'erreur absolue moyenne (MAE).

3. **Entraînement et prédiction** : Le modèle avec les meilleurs paramètres est ensuite entraîné sur l'ensemble d'entraînement complet, et les prédictions sont effectuées sur les données de validation et de test. Un fichier de soumission est généré avec les prédictions sur les données de test.

Cette approche vous permet de tester différentes configurations du modèle et de choisir la meilleure.